## Data Cleaning Accidents BCN:
#### In this Jupyter Notebook you will find the process of data cleaning of a dataset from Ajuntament de Barcelona Open Data on vehicles involved in accident.

First we import the libraries we are going to use.

In [1]:
import numpy as np
import pandas as pd
import re

Import data and assign to variable 'acc' which stands for accident.

In [2]:
acc = pd.read_csv('../Data/Raw_data/vehicles_accident.csv', encoding='utf-8', sep = ';',dtype = 'object')

I examine the data that I have imported.

In [3]:
acc.sample(5)

,Codi_expedient,Codi_districte,Nom_districte,Codi_barri,Nom_barri,Codi_carrer,Nom_carrer,Num_postal,Descripcio_dia_setmana,Dia_setmana,...,Descripcio_tipus_vehicle,Descripcio_model,Descripcio_marca,Descripcio_color,Descripcio_carnet,Antiguitat_carnet,Coordenada_UTM_X,Coordenada_UTM_Y,Longitud,Latitud
45100,2016S001560,2,Eixample,7,la Dreta de l'Eixample,247659,Pau Claris,9999 9999,Diumenge,Dg,...,Taxi,OCTAVIA,SKODA,Altres,BTP,33,"430023,88","4583353,85","2,161784","41,396826"
83668,2014S008346,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",198203,Marquès de l'Argentera,0001 0001,Dijous,Dj,...,Bicicleta,Desconegut,Desconegut,Desconegut,Desconegut,Desconegut,"4581836,63","431836,96",NaN,NaN
17303,2018S001902,2,Eixample,7,la Dreta de l'Eixample,305100,ProvenÃ§a / Sant Joan ...,NaN,Dijous,Dj,...,Bicicleta,Desconegut,Desconegut,Desconegut,Desconegut,Desconegut,430599.12,4583712.76,2.168.634,41.400.116
10629,2018S008466,4,Les Corts,20,la Maternitat i Sant Ramon,23800,Arizala ...,NaN,Dilluns,Dl,...,Motocicleta,XSR 700,YAMAHA,Negre,A1,2,426803.69,4581349.76,212.352,41.378.497
54326,2016S001356,8,Nou Barris,50,les Roquetes,120807,Favència,0199 0199,Diumenge,Dg,...,Furgoneta,JUMPER,CITROEN,Blanc,B,10,"431237,9","4588727,86","2,175693","41,445332"


As I want to study which common characteristics the vehicles involved in an accident have, there are some columns that I won't use, so I will drop them.

In [4]:
acc.columns #Checking the names of the columns.

Index(['Codi_expedient', 'Codi_districte', 'Nom_districte', 'Codi_barri',
       'Nom_barri', 'Codi_carrer', 'Nom_carrer', 'Num_postal ',
       'Descripcio_dia_setmana', 'Dia_setmana', 'Descripcio_tipus_dia', 'Any',
       'Mes_any', 'Nom_mes', 'Dia_mes', 'Hora_dia', 'Descripcio_torn',
       'Descripcio_causa_vianant', 'Descripcio_tipus_vehicle',
       'Descripcio_model', 'Descripcio_marca', 'Descripcio_color',
       'Descripcio_carnet', 'Antiguitat_carnet', 'Coordenada_UTM_X',
       'Coordenada_UTM_Y', 'Longitud', 'Latitud'],
      dtype='object')

In [5]:
#Dropping columns that I will not be using.
acc.drop(columns = ['Codi_districte', 'Nom_districte', 'Codi_barri', \
                    'Nom_barri', 'Codi_carrer', 'Nom_carrer', 'Num_postal ', \
                    'Descripcio_dia_setmana', 'Dia_setmana', 'Descripcio_tipus_dia', 'Any', \
                    'Mes_any', 'Nom_mes', 'Dia_mes', 'Hora_dia', 'Descripcio_torn',\
                    'Descripcio_causa_vianant','Descripcio_carnet','Coordenada_UTM_X',\
                    'Coordenada_UTM_Y', 'Longitud', 'Latitud','Descripcio_model' ], inplace = True)

I will now change the name of the columns so that they are more self-explanatory.

In [6]:
acc.rename(columns = {'Codi_expedient': 'Accident_code', 'Descripcio_tipus_vehicle':'Vehicle_type', 
                      'Descripcio_marca':'Brand','Descripcio_color':'Color', 'Antiguitat_carnet':'Years_license'},
          inplace = True)

Checking datatypes. I will change the type of Years_license at the end, since it has some values as Desconegut (Unknown) that don't let me change directly the data type.

In [7]:
acc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96125 entries, 0 to 96124
Data columns (total 5 columns):
Accident_code    96124 non-null object
Vehicle_type     96124 non-null object
Brand            96124 non-null object
Color            96124 non-null object
Years_license    96124 non-null object
dtypes: object(5)
memory usage: 3.7+ MB


##### Vehicle_type column:

Moreover, I will change the values of the columns Vehicle_type and Color to make it more understandable. First, I will explore which are the values of Vehicle_Type.

In [8]:
acc['Vehicle_type'].value_counts() #Checking the values of Vehicle_type.

Motocicleta                            30293
Turismo                                25908
Turisme                                13616
Furgoneta                               6128
Ciclomotor                              5332
Taxi                                    5092
Bicicleta                               3849
Autobús                                 1995
Camión <= 3,5 Tm                         952
AutobÃºs                                 476
Camión > 3,5 Tm                          423
CamiÃ_ rÃ_gid <= 3,5 tones               280
Camió rígid <= 3,5 tones                 208
CamiÃ_ rÃ_gid > 3,5 tones                169
Autocar                                  161
Tractocamión                             150
Todo terreno                             148
Veh. mobilitat personal amb motor        128
Camió rígid > 3,5 tones                   98
Otros vehíc. a motor                      86
Tot terreny                               80
Altres vehicles amb motor                 62
AutobÃºs a

Since I am studying the common elements between accidents, I will get rid of all the values except for cars and bikes. First, I will unify the names to Cars and bikes and then I will get rid of the other types of vehicles.

In [9]:
acc['Vehicle_type'].replace({'Turisme':'Car', 'Turismo':'Car', 'Todo terreno':'Car', \
                             'Tot terreny':'Car','Motocicleta':'Bike','Ciclomotor':'Bike'}, inplace = True)

Now I am getting rid of the other types of vehicles different than Car and Bike.

In [10]:
#Updating variable acc since it is the only information that we will be using.
acc = acc.query('Vehicle_type == "Car" | Vehicle_type == "Bike"')

##### Brand Column

Let's move to the column Brand. We can see that there are many brands with points and spaces, so I will get rid of them.

In [11]:
list(acc['Brand'].unique()) 
#Seeing the unique values for Brands, if I used value_counts, it didn't print all the values.

['SEAT',
 'PEUGEOT',
 'RENAULT',
 'Desconegut',
 'FORD',
 'HONDA',
 'SYM',
 'BMW',
 'A.U.D.I.',
 'OPEL',
 'PIAGGIO',
 'KIA',
 'YAMAHA',
 'VOLKSWAGEN',
 'SKODA',
 'APRILIA',
 'HARLEY DAVIDSON',
 'MERCEDES',
 'EMCO',
 'KAWASAKI',
 'LANCIA',
 'AUDI',
 'KYMCO',
 'FIAT',
 'TOYOTA',
 'MAZDA',
 'KEEWAY',
 'CITROEN',
 'DAEWOO',
 'KTM',
 'A U D I',
 'DAELIM',
 'S.Y.M',
 'DACIA',
 'DODGE',
 'NISSAN',
 'HYUNDAI',
 'DERBI',
 'SUZUKI',
 'LAND ROVER',
 'GILERA',
 'MONTESA',
 'MINI',
 'HYOSUNG',
 'ALFA ROMEO',
 'GOVECS',
 'SCUTUM',
 'TRIUMPH',
 'SMART',
 'YIYING',
 'VOLVO',
 'JEEP',
 'TATA',
 'LEXUS',
 'HUSQVARNA',
 'PORSCHE',
 'MITSUBISHI',
 'CHEVROLET',
 'BENELLI',
 'MASH',
 'SUMCO',
 'BUELL',
 'B.M.W.',
 'CHRYSLER',
 'VESPA',
 'ES DESCONEIX',
 'Brixton',
 'JAGUAR',
 'B M W',
 'RIEJU',
 'DUCATI',
 'SSANGYONG',
 'BETA',
 'SAAB',
 'SUBARU',
 'HANWAY',
 'INFINITI',
 'COOLTRA',
 'IRISBUS',
 'GAS-GAS',
 'MERCEDES-BENZ',
 'TISONG',
 'MBK',
 'MV AUGUSTA',
 'CPI',
 'GENERIC',
 'GUZZI',
 'WOTTAN',
 'SCOMADI

In [12]:
acc['Brand'] = acc['Brand'].apply(lambda x: x.replace('.','')) #Getting rid of points.

In [13]:
acc['Brand'] = acc['Brand'].apply(lambda x: x.replace(' ','')) #Getting rid of additional spaces.

There are some brands that have more information in the brand name, hence, I will only keep the brand name.

In [14]:
acc['Brand'] = acc['Brand'].apply(lambda x: re.sub('MERCED.*','MERCEDES',x)) #Doing the above explained to MERCEDES.

In [15]:
acc['Brand'] = acc['Brand'].apply(lambda x: re.sub('JAGUAR.*','JAGUAR',x)) #Doing the above explained to JAGUAR.

In [16]:
acc['Brand'].replace({'ESDESCONEIX':'Unknown','Desconegut':'Unknown'}, inplace = True) #Uniforming unknown values.

##### Column Color:

In [17]:
acc['Color'].unique()

array(['Blanc', 'Blau', 'Negre', 'Desconegut', 'Vermell', 'Gris', 'Verd',
       'MarrÃ_', 'Platejat', 'Taronja', 'granate', 'Altres', 'Violeta',
       'Groc', 'Negre/Groc', 'Beige', 'Daurat', 'Rosa', 'Marró'],
      dtype=object)

There are some values that have special characters and I have decided to translate it into English.

In [18]:
acc['Color'].replace({'Blanc':'White', 'Blau':'Blue', 'Negre':'Black', 'Desconegut':'Unknown',
                      'Vermell':'Red', 'Gris':'Grey', 'Verd':'Green','MarrÃ_':'Brown',
                      'Platejat':'Silver', 'Taronja':'Orange', 'granate':'Red', 'Altres':'Unknown',
                      'Violeta':'Purple','Groc':'Yellow', 'Negre/Groc':'Black-Yellow',
                      'Beige':'White', 'Daurat':'Golden', 'Rosa':'Purple', 'Marró':'Brown'}, inplace = True)

##### Column Years_license:

In [19]:
acc['Years_license'].unique()

array(['5', '1', '2', 'Desconegut', '19', '12', '10', '23', '18', '22',
       '4', '11', '13', '17', '21', '29', '28', '14', '8', '15', '32',
       '34', '7', '46', '38', '6', '3', '9', '20', '50', '0', '25', '16',
       '27', '33', '39', '40', '24', '35', '41', '36', '52', '31', '30',
       '26', '51', '37', '58', '48', '45', '42', '44', '43', '54', '47',
       '60', '56', '53', '55', '49', '57', '61', '64', '63', '65', '68',
       '59', '66', '71', '67', '62', '-46', '69', '-189', '73', '-5',
       '-47', '-2', '1818', '-484', '-972', '-70', '-947', '-41', '-48',
       '-45', '-28', '1018', '-969'], dtype=object)

Let's get rid of the 'Desconegut' values, I will first assign them to a -999 values so that I continue having easy track of them.

In [20]:
acc['Years_license'].replace({'Desconegut':'-999'}, inplace = True)

Now, I will change the datatype into integer.

In [21]:
acc['Years_license'] = acc['Years_license'].astype('int')

There are some strange values (negative & >1000 years). Let's examine them:

In [47]:
acc.query('Years_license < 0 | Years_license > 1000')

,Accident_code,Vehicle_type,Brand,Color,Years_license
8,2018S006793,Car,Unknown,Unknown,-999
14,2018S009491,Bike,SYM,White,-999
15,2018S009491,Car,BMW,Grey,-999
58,2018S003738,Bike,HONDA,White,-999
60,2018S003738,Car,Unknown,Unknown,-999
...,...,...,...,...,...
96093,2014S001775,Bike,Unknown,Unknown,-999
96099,2014S008681,Bike,HONDA,Black,-999
96104,2014S008826,Car,HYUNDAI,Green,-999
96109,2014S000418,Car,MAZDA,Grey,-999


Some information seems to be alright so, I will set all the negative values to -999 and also the >1000.

In [43]:
acc_1 = acc.query('Years_license < 0 | Years_license > 1000') #Slice of original dataframe to perform the change.

In [44]:
acc_1['Years_license']=-999 #Setting Years_license to -999

/usr/local/Cellar/ipython/7.5.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [53]:
acc.update(acc_1) #Updating values of acc_1 to acc.

In [55]:
acc.query('Years_license < 0 | Years_license > 1000')['Years_license'].value_counts() 
#Checking if values have changed

-999.0    6835
Name: Years_license, dtype: int64

Let's check if some columns have at the same time Unknown, in that case, we would eliminate them

In [56]:
len(acc.query('Brand == "Unknown" & Color == "Unknown" & Years_license < 0'))*100/len(acc)

0.7840587977765101

0.78% of the acc dataframe has Unknown values, I will now delete them.

In [63]:
acc = acc.query('Brand != "Unknown" & Color != "Unknown" & Years_license >= 0')

Now acc has the information clean.

In [66]:
acc.to_csv('../data/Clean_data/clean_vehicles_bcn.csv')